Import Packages

In [209]:
import pandas as pd
import numpy as np
import warnings

from datetime import datetime, timedelta
from IPython.display import display
# !pip install -U pandasql
from pandasql import sqldf
mysql = lambda q: sqldf(q, globals())

warnings.filterwarnings("ignore")

Read Country Data

In [210]:
country_data_processed = pd.read_csv("Country_Processed_Table.csv")#read input data
country_data_processed.head(20)

,Country,Year,Country_Key,Continent,Region,Capital,Currency,Birth_Rate_Crude_Per_1000,Population_Total_Count,Death_Rate_Crude_Per_1000,GNI_Per_Capita_USD,Labour_Force_Female_Percent,Labour_Force_Total_Count,Age_Dependency_Ratio_Percent_Working_Age,Age_First_Marriage_Male,Age_First_Marriage_Female,Age_Dependency_Ratio_Old,Age_Dependency_Ratio_Young,Completeness_Birth_Registration_Percent
0,Canada,2005,0,North America,North America,Ottawa,CAD,10.6,32243753,7.1,34810,46.637376,17821886,44.495093,28.7,26.98,18.954219,25.540874,100.0
1,Canada,2006,1,North America,North America,Ottawa,CAD,10.9,32571174,7.0,38510,46.875172,18030705,44.236994,28.6,26.60,19.138346,25.098648,100.0
2,Canada,2007,2,North America,North America,Ottawa,CAD,11.2,32889025,7.1,43090,47.036713,18379605,44.034776,28.7,26.98,19.353443,24.681333,100.0
3,Canada,2008,3,North America,North America,Ottawa,CAD,11.3,33247118,7.2,45650,46.974754,18667574,43.933903,28.7,26.98,19.620978,24.312924,100.0
4,Canada,2009,4,North America,North America,Ottawa,CAD,11.3,33628895,7.1,43230,47.200377,18799155,43.976753,28.7,26.98,19.966597,24.010156,100.0
5,Canada,2010,5,North America,North America,Ottawa,CAD,11.1,34004889,7.0,44490,47.310514,19001081,44.179945,28.7,26.98,20.406725,23.773220,100.0
6,Canada,2011,6,North America,North America,Ottawa,CAD,11.0,34339328,7.1,47190,47.267516,19147395,44.647634,28.6,26.90,20.903344,23.744289,100.0
7,Canada,2012,7,North America,North America,Ottawa,CAD,11.0,34714222,7.1,51080,47.392995,19322866,45.139185,28.7,26.98,21.486257,23.652928,100.0
8,Canada,2013,8,North America,North America,Ottawa,CAD,10.8,35082954,7.2,52800,47.466474,19546552,45.706975,28.8,27.10,22.150152,23.556824,100.0
9,Canada,2014,9,North America,North America,Ottawa,CAD,10.8,35437435,7.3,52200,47.326076,19629145,46.380541,28.7,26.98,22.869567,23.510974,100.0


Create Fact Table and Add Month Data

In [211]:
fact_table = pd.DataFrame()

month_data_processed = pd.read_csv("Month_Processed_Table.csv")#read input data

fact_table = month_data_processed.merge(country_data_processed, on="Year")
fact_table = fact_table.sort_values('Country')

fact_table.head()


,Month_Key,Name,Month_Num,Quarter,Year,Decade,Country,Country_Key,Continent,Region,...,Death_Rate_Crude_Per_1000,GNI_Per_Capita_USD,Labour_Force_Female_Percent,Labour_Force_Total_Count,Age_Dependency_Ratio_Percent_Working_Age,Age_First_Marriage_Male,Age_First_Marriage_Female,Age_Dependency_Ratio_Old,Age_Dependency_Ratio_Young,Completeness_Birth_Registration_Percent
1491,165,October,10,4,2018,21,Bolivia,109,South America,South America,...,6.465,3840,39.209263,133000453,47.947297,26.675,22.44,8.665518,39.281779,71.9
1572,174,July,7,3,2019,21,Bolivia,110,South America,South America,...,6.510,4050,39.491377,136202238,47.635083,26.675,22.44,8.932369,38.702714,68.0
357,39,April,4,2,2008,20,Bolivia,99,South America,South America,...,6.849,1940,37.966295,110890081,51.508237,26.675,22.44,7.503345,44.004892,68.0
996,110,March,3,1,2014,21,Bolivia,105,South America,South America,...,6.437,3620,37.975372,122770335,49.357239,26.675,22.44,7.883945,41.473294,68.0
159,17,June,6,2,2006,20,Bolivia,97,South America,South America,...,7.087,1380,35.879524,103776784,52.414299,26.675,22.44,7.419868,44.994431,68.0


Join with Population Data

In [212]:
population_data_processed = pd.read_csv("Pop_Processed_Table.csv")#read input data

fact_table = fact_table.merge(population_data_processed, left_on=['Country','Year'], right_on=['Country','Year'], how='left' )

fact_table.head()

,Month_Key,Name,Month_Num,Quarter,Year,Decade,Country,Country_Key,Continent,Region,...,Population_Growth_Percent,Population_Female_Percent,Population_Male_Percent,Rural_Population_Percent,Rural_Population_Growth_Percent,Urban_Population_Percent,Urban_Population_Growth_Percent,Life_Exp_Birth_Total_Yr,Life_Exp_Birth_Male_Yr,Life_Exp_Birth_Female_Yr
0,165,October,10,4,2018,21,Bolivia,109,South America,South America,...,1.421891,49.783399,50.216601,30.575,0.299854,69.425,1.920061,71.239,68.434,74.209
1,174,July,7,3,2019,21,Bolivia,110,South America,South America,...,1.399133,49.799706,50.200294,30.227,0.254399,69.773,1.899151,71.513,68.684,74.508
2,39,April,4,2,2008,20,Bolivia,99,South America,South America,...,1.691118,49.668326,50.331674,34.465,0.385311,65.535,2.384748,66.937,64.656,69.334
3,110,March,3,1,2014,21,Bolivia,105,South America,South America,...,1.544977,49.723902,50.276098,31.953,0.461741,68.047,2.057712,69.891,67.271,72.655
4,17,June,6,2,2006,20,Bolivia,97,South America,South America,...,1.751711,49.659745,50.340255,35.372,0.465219,64.628,2.462901,65.853,63.680,68.137


Remove Unecessary Columns and Add Measures

In [213]:
#keep only keys and columns needed for joins
fact_table = pd.DataFrame(fact_table, columns=['Month_Key', "Country_Key", "Population_Key", "Country", "Year", "Name", "Month_Num"])

population_human_development_raw = pd.read_csv("human_development_index_raw.csv")#read input data


countries = ["Canada", "United States", "Mexico", "Burundi", "Chad", "Niger", "Bolivia", "Philippines", "Indonesia"]

population_human_development_processed = population_human_development_raw[population_human_development_raw["Entity"].isin(countries)]
population_human_development_processed = population_human_development_processed[population_human_development_processed["Year"] >2004]
population_human_development_processed = population_human_development_processed.drop(["Code"], axis=1)

country = "Canada"
for country in countries:
    for i in range(3):
        country_data = {}
        country_data["Entity"] = country
        country_data["Year"] = 2018+i
        population_human_development_processed = population_human_development_processed.append(pd.DataFrame([country_data]), ignore_index=True)


population_human_development_processed['Entity'].replace({"United States": "USA"}, inplace = True)
population_human_development_processed = population_human_development_processed.sort_values(["Entity","Year"], ascending=[True,True])
population_human_development_processed.fillna(method="ffill", inplace=True)#perhaps change to extrapolation

fact_table = fact_table.merge(population_human_development_processed, left_on=['Country','Year' ], right_on=['Entity','Year' ], how='left').drop(columns=["Entity"])
fact_table = fact_table.sort_values(["Country","Year"], ascending=[True,True])
display(fact_table.tail(50))
print(fact_table.shape)

,Month_Key,Country_Key,Population_Key,Country,Year,Name,Month_Num,Human Development Index (UNDP)
1725,134,27,27,USA,2016,March,3,0.922
1726,137,27,27,USA,2016,June,6,0.922
1612,148,28,28,USA,2017,May,5,0.924
1617,147,28,28,USA,2017,April,4,0.924
1620,146,28,28,USA,2017,March,3,0.924
1625,145,28,28,USA,2017,February,2,0.924
1628,144,28,28,USA,2017,January,1,0.924
1637,149,28,28,USA,2017,June,6,0.924
1647,155,28,28,USA,2017,December,12,0.924
1651,154,28,28,USA,2017,November,11,0.924


(1728, 8)


Add Events

In [214]:
events_processed = pd.read_csv("Event_Processed_Table.csv")#read input data
# events_processed.head(30)

# query = '''
# Select * 
# FROM fact_table, event_processed_table
# INNER JOIN
# ON (fact_table.Country = events_processed.Country)
# where
# fact_table.Month_Num >= events_processed.Start_Month
# and fact_table.Month_Num <= events_processed.End_Month
# and fact_table.Month_Num >= events_processed.Start_Year
# and fact_table.Month_Num <= events_processed.End_Year
# '''

query = '''
Select * 
FROM fact_table
outer join events_processed
where
fact_table.Month_Num >= events_processed.Start_Month
'''
queryTwo = '''
Select * 
FROM fact_table 
left join events_processed

on
fact_table.Country = events_processed.Country
and fact_table.Month_Num >= events_processed.Start_Month
and fact_table.Month_Num <= events_processed.End_Month
and fact_table.Year >= events_processed.Start_Year
and fact_table.Year <= events_processed.End_Year
'''

out = mysql(queryTwo)#76464
display(out.head(50))
print(out.shape)


,Month_Key,Country_Key,Population_Key,Country,Year,Name,Month_Num,Human Development Index (UNDP),Event_Key,Year,...,Disaster_Subtype,Country,Start_Month,End_Month,Start_Year,End_Year,Total_Deaths,Total_Injuries,Total_Homeless,Total_Affected
0,6,96,96,Bolivia,2005,July,7,0.624,NaN,NaN,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,96,96,Bolivia,2005,March,3,0.624,NaN,NaN,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,96,96,Bolivia,2005,August,8,0.624,NaN,NaN,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,96,96,Bolivia,2005,May,5,0.624,NaN,NaN,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,96,96,Bolivia,2005,April,4,0.624,NaN,NaN,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,96,96,Bolivia,2005,June,6,0.624,NaN,NaN,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,8,96,96,Bolivia,2005,September,9,0.624,NaN,NaN,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,10,96,96,Bolivia,2005,November,11,0.624,NaN,NaN,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0,96,96,Bolivia,2005,January,1,0.624,NaN,NaN,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,11,96,96,Bolivia,2005,December,12,0.624,NaN,NaN,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(1738, 22)
